In [ ]:
# TODO:
# explore the dataset, check which variables are connected with seurvivability, check corelation, maaybe reduce dimensions
# choose columns
# add cross validation mechanism
# check several classifiers
# take the best  one and submit

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.head()

In [ ]:
df_train = df.drop(columns=['Name','SibSp','Parch','Ticket','Embarked','Cabin'])
df_train.head()

In [ ]:
def cvt_sex(row):
    if row['Sex'] == 'male':
        return 1.0
    if row['Sex'] == 'female':
        return 0.0

In [ ]:
df_train['Sex'] = df_train.apply(lambda row: cvt_sex(row),axis=1)

In [ ]:
df_train['Age'] = df_train["Age"].fillna(df_train['Age'].mean())
df_train["Fare"] = df_train["Fare"].fillna(df_train['Fare'].mean())

In [ ]:
df_train["Sex"].isna().sum()
df_train["Fare"].isna().sum()

In [ ]:
X = df_train.drop(columns = ['Survived','PassengerId']).values
y = df_train['Survived'].values

In [ ]:
from sklearn import preprocessing

X = preprocessing.MinMaxScaler().fit_transform(X)

In [ ]:
X

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


cross_val_scores = {'NeuralNetwork':0.0,'RandomForest':0.0,'DecisionTree':0.0,'LR':0.0, 'SVM':0.0,'KNN':0.0}
clfs = {'NeuralNetwork':MLPClassifier(random_state=1, max_iter=3000),'RandomForest':RandomForestClassifier(max_depth=10, random_state=5),
        'DecisionTree':DecisionTreeClassifier(random_state=0),'LR':LogisticRegression(random_state=0), 'SVM':SVC(gamma=2, C=1),'KNN':KNeighborsClassifier(n_neighbors=2)}

for key in cross_val_scores:
    cross_val_scores[key] = cross_val_score(clfs[key], X, y, cv=10)

In [ ]:
 for key in cross_val_scores:
        print("Classifier: {}".format(key))
        print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_scores[key].mean(), cross_val_scores[key].std() * 2))

In [ ]:
pd.read_csv('/kaggle/input/titanic/test.csv').head()

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data['Sex'] = test_data.apply(lambda row: cvt_sex(row),axis=1)
test_data = test_data.drop(columns=['Name','SibSp','Parch','Ticket','Embarked','Cabin'])
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())
test_data_vals = preprocessing.MinMaxScaler().fit_transform(test_data)

In [ ]:
test_data_vals[:,1:]

In [ ]:
clf = RandomForestClassifier(max_depth=10, random_state=5)
clf.fit(X, y)

predictions = clf.predict(test_data_vals[:,1:])

In [ ]:
predictions

In [ ]:
submission = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':predictions})

In [ ]:
submission.describe()

In [ ]:
filename = 'TitanicPredictionRandomForestsJP.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)